# Chains in LangChain

- LLMChain
- Sequential Chains
    - SimpleSequentialChain
    - SequentialChain
- Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('../data/Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is a good name to describe \
    a company that makes {product}?"
)

In [9]:
llm = ChatOpenAI(model=llm_model, temperature=0.7)
chain = LLMChain(llm=llm, prompt=prompt)


In [10]:
chain.run(product="Playstation 5")

'Sony Interactive Entertainment.'

In [13]:
chain.run("Chocolate Jelly Beans")

'ChocoJelly Co.'

## SimpleSequentialChain

In [16]:
from langchain.chains import LLMChain, SimpleSequentialChain

In [15]:
llm = ChatOpenAI(model=llm_model, temperature=0.7)


In [33]:
first_prompt = ChatPromptTemplate.from_template(
    "What is a good name to describe \
    a company which makes {product}"
)

first_chain = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    """Write a 20 word description for the following \
    company: {company_name}
    """
)

second_chain = LLMChain(llm=llm, prompt=second_prompt)

In [34]:
overall_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain]
)

In [38]:
overall_chain.run("Playstation")

'"Leading video game company, producing popular titles for the PlayStation console and providing immersive gaming experiences for players worldwide."'

## SequentialChain

In [43]:
from langchain.chains import LLMChain, SequentialChain

In [40]:
llm = ChatOpenAI(model=llm_model, temperature=0.7)

In [48]:
first_prompt = ChatPromptTemplate.from_template("""
Translate the following review into English
{review}
""")

first_chain = LLMChain(llm=llm, prompt=first_prompt, output_key="english_review")

In [49]:
second_prompt = ChatPromptTemplate.from_template("""
Summarize the following review:
{english_review}
"""
)
second_chain = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [50]:
third_prompt = ChatPromptTemplate.from_template("""
Identify the language in the following review:
{review}
""")

third_chain = LLMChain(llm=llm, prompt=third_prompt, output_key="language")


In [53]:
fourth_prompt = ChatPromptTemplate.from_template("""
Write a followup response to the following:

review: {review} in language: {language}
"""
)

fourth_chain = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_response")

In [54]:
final_chain = SequentialChain(
    chains=[first_chain, second_chain, third_chain, fourth_chain],
    input_variables=["review"],
    output_variables=["english_review", "language", "summary", "followup_response"]
)

In [57]:
review = df.Review[5]
response = final_chain(review)


In [58]:
response.keys()

dict_keys(['review', 'english_review', 'language', 'summary', 'followup_response'])

In [60]:
pd.DataFrame([response]).T

,0
review,Je trouve le goût médiocre. La mousse ne tient...
english_review,I find the taste mediocre. The foam doesn't ho...
language,French.
summary,The reviewer does not like the taste of the pr...
followup_response,"Bonjour,\n\nNous sommes désolés d'apprendre qu..."


## RouterChain

In [91]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

### Define Template Prompts



In [92]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [93]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

### Setup Destination Chains

In [94]:
llm = ChatOpenAI(temperature=0.7, model=llm_model)

In [95]:
destination_chains = {}

for info in prompt_infos:
    name = info["name"]
    desc = info["description"]
    prompt = ChatPromptTemplate.from_template(info['prompt_template'])
    destination_chains[name] = LLMChain(llm=llm, prompt=prompt)

destination_prompt_str = '\n'.join([f"{info['name']}: {info['description']}" for info in prompt_infos])
print(destination_prompt_str)

physics: Good for answering questions about physics
math: Good for answering math questions
history: Good for answering history questions
computer science: Good for answering computer science questions


In [96]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [97]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [98]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [99]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destination_prompt_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [100]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [101]:
chain.run("Explain E=mc^2")



> Entering new MultiPromptChain chain...
physics: {'input': 'Explain the equation E=mc^2'}
> Finished chain.


'The equation E=mc^2 is one of the most famous equations in physics, and it relates to the relationship between energy and mass. In this equation, E stands for energy, m stands for mass, and c stands for the speed of light. The equation states that energy and mass are interchangeable and can be converted from one to the other.\n\nSpecifically, the equation states that the energy (E) of an object is equal to its mass (m) times the speed of light (c) squared. This means that even a small amount of mass can contain a huge amount of energy, as the speed of light is a very large number. \n\nThis equation was first proposed by Albert Einstein in his theory of special relativity and has been confirmed through numerous experiments and observations since then. It has important implications for nuclear physics and the study of the universe as a whole.'

In [102]:
chain("What is 5 + 5")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 5 + 5'}
> Finished chain.


{'input': 'What is 5 + 5', 'text': 'The answer to 5 + 5 is 10.'}

In [103]:
chain("What is the role of the pineal gland in human body?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the role of the pineal gland in human body?'}
> Finished chain.


{'input': 'What is the role of the pineal gland in human body?',
 'text': "The pineal gland is a small endocrine gland located in the brain that produces and secretes the hormone melatonin. Melatonin plays a role in regulating the body's natural sleep-wake cycle (circadian rhythm) and is involved in the regulation of several physiological processes, including mood, immune function, and reproductive function. The pineal gland also has a role in regulating the body's response to light and is involved in the production of serotonin, a neurotransmitter that plays a role in mood regulation. Some studies suggest that the pineal gland may also play a role in the regulation of aging and longevity."}

In [104]:
chain("Explain transformer neural networks")



> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain transformer neural networks in the context of natural language processing'}
> Finished chain.


{'input': 'Explain transformer neural networks in the context of natural language processing',
 'text': 'Transformer neural networks are a type of neural network architecture that has been widely used in natural language processing tasks. They were first introduced in a 2017 paper by Vaswani et al. and have since become one of the most popular approaches to NLP.\n\nThe key innovation of transformer neural networks is the use of self-attention mechanisms. In traditional neural networks, each input token is processed independently and sequentially, which can be computationally expensive and make it difficult to capture long-range dependencies between tokens.\n\nHowever, in a transformer network, each input token is processed simultaneously, with attention weights assigned to each token based on its relationships with other tokens in the sequence. This allows the network to effectively capture global dependencies between tokens, leading to improved performance on tasks such as machine tra